# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
#pip install gmaps

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from ast import literal_eval
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Store filepath in a variable
vac_df =pd.read_csv("Weather_output.csv")
#vac_df = "Weather_output.csv"
vac_df.head()
#file_read = pd.read_csv(file_one)
#vac_df = pd.DataFrame(file_read)
#vac_df_new = vac_df[vac_df['message'].notnull()]
#vac_df = vac_df.dropna(how="all")
#vac_df.head()
#pprint(vac_df)
#vac_df = pd.read_csv('Weather_output.csv')
#vac_df.dropna()
#vac_df.head()

,Unnamed: 0,coord,weather,base,main,wind,snow,clouds,dt,sys,timezone,id,name,cod,visibility,rain,message
0,0,"{'lon': 81.47, 'lat': 50.99}","[{'id': 600, 'main': 'Snow', 'description': 'l...",model,"{'temp': 6.46, 'feels_like': -8.55, 'temp_min'...","{'speed': 15.84, 'deg': 260}",{'3h': 0.38},{'all': 100},1.580252e+09,"{'country': 'RU', 'sunrise': 1580264180, 'suns...",25200.0,1506260.0,Gornyak,200,NaN,NaN,NaN
1,1,"{'lon': -147.72, 'lat': 64.84}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': -22.4, 'feels_like': -31.22, 'temp_mi...","{'speed': 3.27, 'deg': 19}",NaN,{'all': 1},1.580252e+09,"{'type': 1, 'id': 7684, 'country': 'US', 'sunr...",-32400.0,5861897.0,Fairbanks,200,16093.0,NaN,NaN
2,2,"{'lon': -165.41, 'lat': 64.5}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': -18.4, 'feels_like': -28.62, 'temp_mi...","{'speed': 5.82, 'deg': 20}",NaN,{'all': 1},1.580252e+09,"{'type': 1, 'id': 7770, 'country': 'US', 'sunr...",-32400.0,5870133.0,Nome,200,16093.0,NaN,NaN
3,3,"{'lon': 45.76, 'lat': 64.9}","[{'id': 801, 'main': 'Clouds', 'description': ...",model,"{'temp': -7.53, 'feels_like': -16.65, 'temp_mi...","{'speed': 4.47, 'deg': 188}",NaN,{'all': 21},1.580252e+09,"{'country': 'RU', 'sunrise': 1580277341, 'suns...",10800.0,535839.0,Leshukonskoye,200,NaN,NaN,NaN
4,4,"{'lon': 15.74, 'lat': -12.78}","[{'id': 500, 'main': 'Rain', 'description': 'l...",model,"{'temp': 60.71, 'feels_like': 62.51, 'temp_min...","{'speed': 2.15, 'deg': 109}",NaN,{'all': 100},1.580252e+09,"{'country': 'AO', 'sunrise': 1580186933, 'suns...",3600.0,3348313.0,Huambo,200,NaN,{'3h': 1.06},NaN


In [13]:
#vac_df = vac_df[['coord'][index]["lat"]]
       # x.append(weather_results_df['main'][index]['temp'])
       

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
#* **Note:** if you are having trouble displaying the maps try running `jupyter nbextension enable --py gmaps` in your environment and retry.
# loop 

lat=[]
lon=[]
hum=[]

#print(vac_df['coord'][1]['lat'])

for index, weatherstuff in vac_df.iterrows():
    try:
        coord_obj = literal_eval(vac_df['coord'][index])
        main_obj = literal_eval(vac_df['main'][index])
        lat.append(coord_obj['lat'])
        lon.append(coord_obj['lon'])
        hum.append(main_obj['humidity'])
    except Exception:
            continue

#print(lat)   

humidity_df = pd.DataFrame(hum).astype(float)
#humidity_df

#locations =pd.DataFrame[['lat', 'lng']]
locations = pd.DataFrame({'lat' : lat, 'lon' : lon}).astype(float)
#print(locations)

# Plot Heatmap
country = [81.47, 50.99]
fig = gmaps.figure(zoom_level=1.5, center= country)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_df, 
                                dissipating=False, max_intensity=100,
                                point_radius=2)
#print(heat_layer)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

# lower than 80, higher than 70
perf_temp = vac_df(['main'][index]["temp"]).between (70,80, inclusive =False)

#wind less than 10
perf_speed = vac_df(['wind'][index]["speed"]<10

#zero clouds
perf_sky = vac_df(['clouds'][index]['all'])=0

# Convert to DataFrame
                    
vac_df = pd.DataFrame(vac_df)
vac_df.dropna()
vac_df


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#3-10
hotel_df = vac_df

In [7]:
# Show mulitple specific columns--note the extra brackets
#hotel_df["perf_temp", "perf_speed", "perf_sky"]].head()


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel name: {hotel}" for hotel in hotel_info]
)

fig = gmaps.figure()
fig.add_layer(marker_layer)

fig